Importar requisitos MCAO
=====================

Cargamos requisitos de MCAO en cosmoSys-Req.

Comenzamos conectándonos a la instancia de cosmoSys-Req a donde importaremos los requisitos, y listamos los proyectos activos.

Utilizamos el fichero configfile_req.py donde guardaremos las opciones de conexión de cosmoSys-Req y otras variables de configuración necesarias para que el código funcione.


In [16]:
from cfg.configfile_req import *
from redminelib import Redmine

redmine = Redmine(req_server_url,key=req_key_txt)
projects = redmine.project.all()

print("Proyectos:")
for p in projects:
    print ("\t",p.identifier," \t| ",p.name)

    

Proyectos:
	 base  	|  Base
	 reqtest  	|  ReqTest
	 robot  	|  Robot
	 rqdemo  	|  RqDemo


Ahora vamos a seleccionar sólo un proyecto: aquél que tiene el identificador almacenado en configfile_req.py dentro de la variable 'project_id_str':

In [17]:
my_project = redmine.project.get(req_project_id_str)
print ("Obtenemos proyecto: ",my_project.identifier," | ",my_project.name)

Obtenemos proyecto:  rqdemo  |  RqDemo


Vamos a recorrer la hoja de cálculo con los datos de carga especificada en la variable const_file_name del fichero configfile_req.py.  Listaremos los datos cargados.

In [18]:
from pyexcel_ods import get_data

file_path = "." + "/uploading/" + req_upload_file_name

data = get_data(file_path, start_column=req_upload_start_column, column_limit=(req_upload_end_column-req_upload_start_column+1),
                start_row=req_upload_start_row, row_limit=(req_upload_end_row - req_upload_start_row+1))
data

OrderedDict([('Intro',
              [[],
               ['Copy RootDoc tab to as many requirement documents you need in the project'],
               ['In the first row of each tab, fill the document attributes, including the Tab name of the parent'],
               ['Fill the requirements at the table.'],
               ['To set the relationships between requirements, use the RqID'],
               ['The tab Dict is giving some categories needed to automatically create the drop-down lists and the links to the database or the diagrams.'],
               [],
               ['In this example, we have to develop two systems: the SYS and the CHILD.'],
               ['The requierements of each system are described in two documents: the RootDoc for the SYS, and the ChildDoc for the CHILD.'],
               ['ChildDoc’s parent is RootDoc.'],
               [],
               []]),
             ('Dict',
              [[],
               ['', 'Req server URL', 'http://myrequirementsserver.dom

Recorreremos ahora las diferentes pestañas para sacar los documentos de requisitos que debemos crear

In [27]:
for t in data:
    print("********************"+t)
    #primero filtramos la pestaña Intro
    if (t!='Intro') and (t!='Dict'):
        # Obtenemos el contenido de esa pestaña
        dataimport = data[t]
        # Obtenemos la primera fila, donde están los datos adicionales del documento de requisito
        d = dataimport[0]
        # Como título, tomamos la pestaña
        docidstr = t
        print("DocID: "+docidstr)
        # Como descripción, tomamos el contenido de la celda A2
        docname = d[1]
        print("DocName: ",docname)
        prefixstr = d[5]
        # A partir del nombre de la pestaña, miramos si el requisito que representa a ese documento ya existe
        doclist = redmine.issue.filter(project_id=req_project_id_str, subject=docidstr, tracker_id=req_doc_tracker_id)
        if(len(doclist)==0):
            # no existe el requisito asociado a la pestaña, lo creo
            print ("Creando documento ",docidstr)
            thisreq = redmine.issue.create(project_id = req_project_id_str,
                                           tracker_id = req_doc_tracker_id,
                                           subject = docidstr,
                                           description = docname,
                                           custom_fields=[{'id': req_title_cf_id,'value': docname},
                                                          {'id': req_prefix_cf_id, 'value': prefixstr},
                                                          {'id': req_chapter_cf_id, 'value': prefixstr}
                                                         ]
                                          )
        else:
            # existe el requisito, así que lo actualizo
            print ("actualizando documento", doclist[0].id)
            redmine.issue.update(resource_id=doclist[0].id,
                                 description = docname,
                                 custom_fields=[{'id': req_title_cf_id,'value': docname},
                                                {'id': req_prefix_cf_id, 'value':prefixstr},
                                                {'id': req_chapter_cf_id, 'value': prefixstr}
                                               ]
                                 )
            


********************Intro
********************Dict
********************SR
DocID: SR
DocName:  Requirements root document
actualizando documento 272
********************SRC
DocID: SRC
DocName:  Requirements child document
actualizando documento 273


Vamos a buscar los documentos padre de los documentos ya existentes

In [28]:
for t in data:
    print("********************"+t)
    # No vamos a tratar la pestaña "Intro"
    if (t!='Intro') and (t!='Dict'):
        # importamos los datos 
        dataimport = data[t]
        # Buscamos el documento que representa a la pestaña
        doclist = redmine.issue.filter(project_id=req_project_id_str, subject=docidstr, tracker_id=req_doc_tracker_id)
        if(len(doclist)>0):
            # Existe el documento asociado a la pestaña, puedo tratarlo
            thisdoc = doclist[0]
            print("rqid: "+thisdoc.subject)
            d = dataimport[0]
            parentdocstr = d[3]
            print("d: ",d)
            print("parent str: " + parentdocstr)
            if (len(parentdocstr)>0):
                # Se ha especificado un documento padre
                parentdoclist = redmine.issue.filter(project_id=req_project_id_str,subject=parentdocstr, tracker_id=req_doc_tracker_id)
                if len(parentdoclist)>0:
                    # Existe el documento padre, puedo tratarlo
                    parentdoc = parentdoclist[0]
                    print("parent: ",parentdoc)
                    print("parent id:",parentdoc.id)
                    redmine.issue.update(resource_id=thisdoc.id, parent_issue_id = parentdoc.id)
                else:
                    print("error, no encontramos el padre")


********************Intro
********************Dict
********************SR
rqid: SRC
d:  ['DocName->', 'Requirements root document', 'ParentDoc->', '', 'Prefix->', 'SR']
parent str: 
********************SRC
rqid: SRC
d:  ['DocName->', 'Requirements child document', 'ParentDoc->', 'SR', 'Prefix->', 'SRC']
parent str: SR
parent:  SR
parent id: 272


Ya tenemos los documentos de requisitos (pestañas) cargados como requisitos padres.  Ahora puedo recorrer los requisitos de cada una de las pestañas y crear los requisitos que falten.

In [35]:
for t in data:
    print("********************"+t)
    if (t!='Intro') and (t!='Dict'):
        dataimport = data[t]
        reqdoclist = redmine.issue.filter(project_id=req_project_id_str, status_id='*',subject=t)
        print(t, "len list: ",len(reqdoclist))
        if(len(reqdoclist)>0):
            reqDoc = reqdoclist[0]
            print("reqDocId:", reqDoc.id)
            reqDocPrefix = reqDoc.custom_fields.get(req_prefix_cf_id).value
            print("reqDocPrefix:",reqDocPrefix)
            # Obtendremos en un bucle todos los requisitos, saltándonos las 2 líneas iniciales
            firstLine = True
            secondLine = True
            for r in dataimport:
                if firstLine:
                    firstLine = False
                else:
                    if secondLine:
                        secondLine = False
                    else:
                        #print(r)
                        if len(r)<16:
                            print("Saltamos")
                        else:
                            # Estamos procesando las líneas de requisitos
                            rqidstr = r[4]
                            print("rqid: "+rqidstr)
                            related_str = r[10]
                            print("related: "+related_str)
                            title_str = r[5]
                            print("title: ",title_str)
                            descr = r[6]
                            print("description: ",descr)
                            reqsource = r[11]
                            print("reqsource: ",reqsource)
                            reqtype = r[9]
                            print("reqtype: ",reqtype)
                            reqlevel = r[8]
                            print("reqlevel: ",reqlevel)
                            reqrationale = r[15]
                            print("reqrationale: ",reqrationale)
                            reqvar = r[13]
                            print("reqvar: ",reqvar)
                            reqvalue = r[14]
                            print("reqvalue: ",reqvalue)
                            rqchapter = reqDocPrefix+r[0]
                            print("rqchapter: ",rqchapter)
                            rqtarget = r[12]
                            print("rqtarget: ",rqtarget)
                            findVersionSuccess = False
                            thisVersionId = None
                            for v in my_project.versions:
                                if not(findVersionSuccess):
                                    if (v.name == 'rqtarget'):
                                        findVersionSuccess = True
                                        thisVersionId = v.id

                            
                            # A partir del nombre del requisito, miramos si el requisito ya existe
                            reqlist = redmine.issue.filter(project_id=req_project_id_str,subject=rqidstr, tracker_id=req_rq_tracker_id)
                            if(len(reqlist)==0):
                                # no existe el requisito, lo creo
                                print ("creando requisito")
                                thisreq = redmine.issue.create(project_id = req_project_id_str,
                                                               tracker_id = req_rq_tracker_id,
                                                               subject = rqidstr,
                                                               description = descr,
                                                               fixed_version_id = thisVersionId,
                                                               custom_fields=[{'id': req_title_cf_id,'value': title_str},
                                                                              {'id': req_sources_cf_id,'value': reqsource},
                                                                              {'id': req_type_cf_id,'value': reqtype},
                                                                              {'id': req_level_cf_id,'value': reqlevel},
                                                                              {'id': req_rationale_cf_id,'value': reqrationale},
                                                                              {'id': req_var_cf_id,'value': reqvar},
                                                                              {'id': req_value_cf_id,'value': reqvalue},
                                                                              {'id': req_chapter_cf_id, 'value':rqchapter}
                                                                             ]
                                                              )

                                # Creo la pertenencia al documento
                                relation = redmine.issue_relation.create(
                                    issue_id=reqDoc.id,
                                    issue_to_id=thisreq.id,
                                    relation_type='blocks'
                                )
                                
                                
                            else:
                                # existe el requisito, así que lo actualizo
                                print ("actualizando requisito", reqlist[0].id)
                                redmine.issue.update(resource_id=reqlist[0].id,
                                                     description = descr,
                                                     fixed_version_id = thisVersionId,
                                                     custom_fields=[{'id': req_title_cf_id,'value': title_str},
                                                                    {'id': req_sources_cf_id,'value': reqsource},
                                                                    {'id': req_type_cf_id,'value': reqtype},
                                                                    {'id': req_level_cf_id,'value': reqlevel},
                                                                    {'id': req_rationale_cf_id,'value': reqrationale},
                                                                    {'id': req_var_cf_id,'value': reqvar},
                                                                    {'id': req_value_cf_id,'value': reqvalue},
                                                                    {'id': req_chapter_cf_id, 'value':rqchapter}
                                                                    ]
                                                    )


                                # Busco si existe la relación entre el requisito y el documento
                                my_req_relations = redmine.issue_relation.filter(issue_id=reqlist[0].id)
                                # Como voy a tratar las que tienen el requisito como destino, las filtro
                                my_req_dest_relations = list(filter(lambda x: x.issue_to_id == reqlist[0].id, my_req_relations))
                                my_req_doc_relations = list(filter(lambda x: x.issue_id == reqDoc.id, my_req_dest_relations))
                                print("req <- doc relations ",my_req_doc_relations)
                                if (len(my_req_doc_relations)>0):
                                    print("Ya existe la relación, no hacemos nada")
                                else:
                                    # Creo la pertenencia al documento
                                    relation = redmine.issue_relation.create(
                                        issue_id=reqDoc.id,
                                        issue_to_id=thisreq.id,
                                        relation_type='blocks'
                                    )


********************Intro
********************Dict
********************SR
SR len list:  1
reqDocId: 272
reqDocPrefix: SR
rqid: SR0001
related: 
title:  Introduction
description:  This document is the root document for the requirements of the system SYS
reqsource:  
reqtype:  Info
reqlevel:  System
reqrationale:  The document needs a readable structure
reqvar:  
reqvalue:  
rqchapter:  SR1.
rqtarget:  v0.1
actualizando requisito 274
req <- doc relations  [<redminelib.resources.IssueRelation #219>]
Ya existe la relación, no hacemos nada
rqid: SR0002
related: 
title:  About this document
description:  The SYS system has to fulfill the requirements in this document.
reqsource:  
reqtype:  Info
reqlevel:  System
reqrationale:  The document needs a readable structure
reqvar:  
reqvalue:  
rqchapter:  SR1.1
rqtarget:  v0.1
actualizando requisito 275
req <- doc relations  [<redminelib.resources.IssueRelation #220>]
Ya existe la relación, no hacemos nada
rqid: SR0003
related: 
title:  Requireme

req <- doc relations  [<redminelib.resources.IssueRelation #231>]
Ya existe la relación, no hacemos nada
rqid: SRC0004
related: 
title:  Functional requirements
description:  This section defines the functional requirements of the CHILD system.  These functional requirements are the main reason why the CHILD system has to be developed.
reqsource:  
reqtype:  Info
reqlevel:  System
reqrationale:  The document needs a readable structure
reqvar:  
reqvalue:  
rqchapter:  SRC2.1.
rqtarget:  v0.1
actualizando requisito 287
req <- doc relations  [<redminelib.resources.IssueRelation #232>]
Ya existe la relación, no hacemos nada
rqid: SRC0005
related: SR0004
title:  Speed
description:  The system shall operate at an SysSpeed greater than 30 +/- 0.01 fps
reqsource:  RD10
reqtype:  Complex
reqlevel:  System
reqrationale:  The RD10 SR0004 states that the SYS subsystem is operating at that speed.  The CHILD subsystem has tao operate at the same speed.
reqvar:  SysSpeed
reqvalue:  30 +/- 0.01 fps
r

Ahora buscamos las relaciones entre requisitos padres e hijos, y de dependencia.

In [34]:
for t in data:
    print("********************"+t)
    if (t!='Intro') and (t!='Dict'):
        dataimport = data[t]
        reqdoclist = redmine.issue.filter(project_id=req_project_id_str, status_id='*',subject=t, tracker_id=req_doc_tracker_id)
        print(t, "len list: ",len(reqdoclist))
        if(len(reqdoclist)>0):
            reqDoc = reqdoclist[0]
            print("reqDocId:", reqDoc.id)
            # Obtendremos en un bucle todos los requisitos, saltándonos las 2 líneas iniciales
            firstLine = True
            secondLine = True
            for r in dataimport:
                if firstLine:
                    firstLine = False
                else:
                    if secondLine:
                        secondLine = False
                    else:
                        #print(r)
                        if len(r)<16:
                            print("Saltamos una fila que no tiene suficientes celdas")
                        else:
                            # Estamos procesando las líneas de requisitos
                            rqidstr = r[4]
                            print("rqid: "+rqidstr)
                            related_str = r[10]
                            print("related: "+related_str)
                            parent_str = r[7]
                            print("parent_str: ",parent_str)
                            # Accedo al objeto requisito
                            reqlist = redmine.issue.filter(project_id=req_project_id_str,subject=rqidstr, tracker_id=req_rq_tracker_id)
                            if(len(reqlist)>0):
                                # El requisito existe, lo tomo como requisito actual
                                current_req = reqlist[0];
                                if (len(parent_str)>0):
                                    # Buscamos el requisito padre para enlazarlo
                                    parentlist = redmine.issue.filter(project_id=req_project_id_str,subject=parent_str, tracker_id=req_rq_tracker_id)
                                    if len(parentlist)>0:
                                        # Existe el documento padre, puedo tratarlo
                                        parentreq = parentlist[0]
                                        print("parent: ",parentreq)
                                        print("parent id:",parentreq.id)
                                        redmine.issue.update(resource_id=current_req.id, parent_issue_id = parentreq.id)                                    
                                    else:
                                        print("ERROR: No encontramos el requisito padre!!!")
                                        
                                
                                # Exploramos ahora las relaciones de dependencia
                                if (len(related_str)>0):
                                    if (related_str[0]!='-'):
                                        # Busco las relaciones existences con este requisito
                                        my_req_relations = redmine.issue_relation.filter(issue_id=current_req.id)
                                        # Como voy a tratar las que tienen el requisito como destino, las filtro
                                        my_filtered_req_relations = list(filter(lambda x: x.issue_to_id == current_req.id, my_req_relations))
                                        # Ahora saco todos los ID de los requisitos del otro lado (en el lado origen de la relación)
                                        related_req = related_str.split()
                                        # Y voy a recorrer uno a uno
                                        for rreq in related_req:
                                            print("related to: ",rreq)
                                            # Busco ese requisito
                                            blocking_reqlist = redmine.issue.filter(project_id=req_project_id_str,subject=rreq, tracker_id=req_rq_tracker_id)
                                            if (len(blocking_reqlist)>0):
                                                blocking_req = blocking_reqlist[0]
                                                # Veo si ya existe algún tipo de relación con él
                                                preexistent_relations = list(filter(lambda x: x.issue_id == blocking_req.id, my_filtered_req_relations))
                                                print(preexistent_relations)
                                                if (len(preexistent_relations)>0):
                                                    print("Ya existe la relación")
                                                else:
                                                    print("Creo una nueva relación")
                                                    relation = redmine.issue_relation.create(
                                                        issue_id=blocking_req.id,
                                                        issue_to_id=current_req.id,
                                                        relation_type='blocks'
                                                    )


********************Intro
********************Dict
********************SR
SR len list:  1
reqDocId: 272
rqid: SR0001
related: 
parent_str:  
rqid: SR0002
related: 
parent_str:  SR0001
parent:  SR0001
parent id: 274
rqid: SR0003
related: 
parent_str:  
rqid: SR0004
related: 
parent_str:  SR0003
parent:  SR0003
parent id: 276
rqid: SR0005
related: 
parent_str:  SR0004
parent:  SR0004
parent id: 277
rqid: SR0006
related: 
parent_str:  SR0004
parent:  SR0004
parent id: 277
rqid: SR0007
related: SR0005
SR0006
parent_str:  SR0004
parent:  SR0004
parent id: 277
related to:  SR0005
[<redminelib.resources.IssueRelation #239>]
Ya existe la relación
related to:  SR0006
[<redminelib.resources.IssueRelation #240>]
Ya existe la relación
rqid: SR0008
related: SR0005
parent_str:  SR0004
parent:  SR0004
parent id: 277
related to:  SR0005
[<redminelib.resources.IssueRelation #241>]
Ya existe la relación
rqid: SR0009
related: 
parent_str:  SR0003
parent:  SR0003
parent id: 276
rqid: SR0010
related: 
pare

rqid: SRC0002
related: 
parent_str:  SRC0001
parent:  SRC0001
parent id: 284
rqid: SRC0003
related: 
parent_str:  
rqid: SRC0004
related: 
parent_str:  SRC0003
parent:  SRC0003
parent id: 286
rqid: SRC0005
related: SR0004
parent_str:  SRC0004
parent:  SRC0004
parent id: 287
related to:  SR0004
[<redminelib.resources.IssueRelation #242>]
Ya existe la relación
rqid: SRC0006
related: 
parent_str:  SRC0004
parent:  SRC0004
parent id: 287
rqid: SRC0007
related: SRC0005
SRC0006
parent_str:  SRC0004
parent:  SRC0004
parent id: 287
related to:  SRC0005
[<redminelib.resources.IssueRelation #243>]
Ya existe la relación
related to:  SRC0006
[<redminelib.resources.IssueRelation #244>]
Ya existe la relación
rqid: SRC0008
related: SRC0005
parent_str:  SRC0004
parent:  SRC0004
parent id: 287
related to:  SRC0005
[<redminelib.resources.IssueRelation #245>]
Ya existe la relación
rqid: SRC0009
related: 
parent_str:  SRC0003
parent:  SRC0003
parent id: 286
rqid: SRC0010
related: 
parent_str:  SRC0009
par